<a href="https://colab.research.google.com/github/danielmancovidales/Aplicaciones-en-salud/blob/main/tuneo_hiperp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Librerias generales
import pandas as pd
import numpy as np
import re

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
!pip install optuna
import optuna

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Lectura de base de datos
df = pd.read_csv('/content/drive/MyDrive/analitica3/Salud/df_sel.csv')
y = pd.read_csv('/content/drive/MyDrive/analitica3/Salud/y.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
y.drop('Unnamed: 0', axis=1, inplace=True)

In [6]:
#Escalado de la variable target
scaler = MinMaxScaler()
y['tiempo_estancia'] = scaler.fit_transform(y[['tiempo_estancia']])

In [7]:
#separacion de datos
X_train,X_test,y_train,y_test=train_test_split(df,y,test_size=0.3,random_state=0)

# <h1> LGBMRegressor

##**Tuneo**

In [8]:
def objective(trial):
    params = {
        "objective": "regression",
        "metric": "rmse",
        "n_estimators": 1000,
        "verbosity": -1,
        "bagging_freq": 1,
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 2**10),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.001, 1.0, log=True)  # Agrega la regularización L1
    }
    model = lgb.LGBMRegressor(**params)
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    rmse = mean_squared_error(y_test, predictions, squared=False)
    return rmse

In [14]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)

[I 2024-05-07 06:40:04,688] A new study created in memory with name: no-name-bf6f6903-b82b-4eab-8210-9cb098b82263
[I 2024-05-07 06:40:05,599] Trial 0 finished with value: 0.0713968713961589 and parameters: {'learning_rate': 0.0010335467151415744, 'num_leaves': 249, 'subsample': 0.47821834338646835, 'colsample_bytree': 0.2458061834831447, 'min_data_in_leaf': 43, 'reg_alpha': 0.12306484298063528}. Best is trial 0 with value: 0.0713968713961589.
[I 2024-05-07 06:40:06,244] Trial 1 finished with value: 0.07356561144892099 and parameters: {'learning_rate': 0.08801873940450644, 'num_leaves': 207, 'subsample': 0.9476201240238571, 'colsample_bytree': 0.18853610225807477, 'min_data_in_leaf': 71, 'reg_alpha': 0.010938960115638286}. Best is trial 0 with value: 0.0713968713961589.
[I 2024-05-07 06:40:07,323] Trial 2 finished with value: 0.06502016659090612 and parameters: {'learning_rate': 0.0015238723525740937, 'num_leaves': 51, 'subsample': 0.7465166127928877, 'colsample_bytree': 0.7676718502624

In [15]:
print('Best hyperparameters:', study.best_params)
print('Best RMSE:', study.best_value)

Best hyperparameters: {'learning_rate': 0.0026451215825294186, 'num_leaves': 72, 'subsample': 0.7288439007414641, 'colsample_bytree': 0.7855536254656119, 'min_data_in_leaf': 55, 'reg_alpha': 0.09402913388021308}
Best RMSE: 0.06434857099737479


##**Modelo**

In [16]:
#Creacion modelo LGBMRegressor
m_lgb=lgb.LGBMRegressor(learning_rate=0.0026451215825294186, num_leaves=72, subsample= 0.7288439007414641, colsample_bytree= 0.7855536254656119, min_data_in_leaf= 55, reg_alpha= 0.09402913388021308)
m_lgb.fit(X_train,y_train)

[LightGBM] [Warning] min_data_in_leaf is set=55, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=55
[LightGBM] [Warning] min_data_in_leaf is set=55, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=55
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000374 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2652
[LightGBM] [Info] Number of data points in the train set: 906, number of used features: 55
[LightGBM] [Info] Start training from score 0.083391
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive

LGBMRegressor(colsample_bytree=0.7855536254656119,
              learning_rate=0.0026451215825294186, min_data_in_leaf=55,
              num_leaves=72, reg_alpha=0.09402913388021308,
              subsample=0.7288439007414641)

In [17]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Supongamos que ya has ajustado tu modelo y tienes los conjuntos de datos de entrenamiento y prueba
# X_train, X_test, y_train, y_test representan tus datos

# Entrenar tu modelo con X_train y y_train

# Realizar predicciones en el conjunto de entrenamiento
y_pred_train = m_lgb.predict(X_train)

# Realizar predicciones en el conjunto de prueba
y_pred_test = m_lgb.predict(X_test)

# Desescalar los valores predichos y los valores reales tanto para entrenamiento como para prueba
y_train_o = scaler.inverse_transform(y_train)
y_pred_train = y_pred_train.reshape(-1, 1)
y_pred_train_o = scaler.inverse_transform(y_pred_train)

y_test_o = scaler.inverse_transform(y_test)
y_pred_test = y_pred_test.reshape(-1, 1)
y_pred_test_o = scaler.inverse_transform(y_pred_test)

# Calcular métricas R^2, MAE y RMSE para el conjunto de entrenamiento
r2_train = r2_score(y_train_o, y_pred_train_o)
mae_train = mean_absolute_error(y_train_o, y_pred_train_o)
rmse_train = np.sqrt(mean_squared_error(y_train_o, y_pred_train_o))

# Calcular métricas R^2, MAE y RMSE para el conjunto de prueba
r2_test = r2_score(y_test_o, y_pred_test_o)
mae_test = mean_absolute_error(y_test_o, y_pred_test_o)
rmse_test = np.sqrt(mean_squared_error(y_test_o, y_pred_test_o))

# Imprimir los resultados para el conjunto de entrenamiento

print("Train - MAE:", mae_train)
print("Train - RMSE:", rmse_train)


# Imprimir los resultados para el conjunto de prueba

print("Test - MAE:", mae_test)
print("Test - RMSE:", rmse_test)

[LightGBM] [Warning] min_data_in_leaf is set=55, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=55
[LightGBM] [Warning] min_data_in_leaf is set=55, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=55
Train - MAE: 122.37282926154037
Train - RMSE: 192.1954110045669
Test - MAE: 116.24780854010542
Test - RMSE: 164.53630241813036


**Train**

**Antes del tuneo:** Los valores bajos de MAE y RMSE en el conjunto de entrenamiento indicaban que el modelo aprendía patrones de los datos de entrenamiento de manera efectiva.

**Después del tuneo:** Un aumento significativo en MAE y RMSE sugiere que el tuneo de hiperparámetros podría haber causado un sobreajuste del modelo en los datos de entrenamiento. Esto significa que el modelo ahora se adapta demasiado a los detalles específicos del conjunto de entrenamiento y no generaliza bien a nuevos datos no vistos.

**Test**

**Antes del tuneo:** Los valores de MAE y RMSE en el conjunto de prueba eran razonables, lo que indica que el modelo tenía una capacidad decente de generalización a datos no vistos.

**Después del tuneo:** Un ligero aumento en MAE y RMSE en el conjunto de prueba sugiere que el tuneo de hiperparámetros no tuvo un impacto significativo en la capacidad de generalización del modelo.

**Conclusiones**

-El tuneo de hiperparámetros no ha logrado mejorar el rendimiento del modelo en términos de error de entrenamiento o generalización.

-El modelo podría estar sobreajustado a los datos de entrenamiento después del tuneo de hiperparámetros.

-Es posible que los hiperparámetros seleccionados no sean adecuados para el problema específico.

-Los datos quizá no aporten mucho a la explicación de la variable objetivo.

-Para futuros estudios se podría optar por dejar muchas más variables y probar otros modelos ya que la misma naturaleza continua de la variable target hace que sea más compleja su predicción.